#how to setup rtmp server, please reference below url
https://www.cnblogs.com/nowgood/p/ffmpegnginx.html

#send rtmp data from pi4 to server, can use follow command
ffmpeg -f v4l2 -r 45 -video_size vga -pix_fmt yuv420p12be -i /dev/video0 -b:v 500k -c:v h264_omx -preset ultrafast -an -f flv rtmp://ec2-54-222-130-39.cn-north-1.compute.amazonaws.com.cn/live/123456


In [ ]:
!pip install gluoncv

In [ ]:
import boto3
from IPython.display import clear_output, Image, display, HTML
import numpy as np
import cv2
import base64
from bokeh.plotting import figure
from bokeh.io import output_notebook, show, push_notebook
import time
output_notebook()

In [ ]:
# import sagemaker
# sagemaker_endpoint_name = 'object-detection-2019-10-18-17-56-41-925'
# sagemaker_endpoint = sagemaker.predictor.RealTimePredictor(sagemaker_endpoint_name)
# sagemaker_endpoint.content_type = 'image/jpeg'
from object_detection import ObjectDetection
objectDetection = ObjectDetection()

In [ ]:
class VideoPlayer(object):
    def __init__(self):
        self._init = False
        self._myImage = None
        
    def __call__(self, frame):
        if frame is None:
            return
        if self._init is False:
            self.init_display(frame)
            self._init = True
        else:
            self.update_display(frame)

    def init_display(self, frame):
        assert frame is not None
        frame=cv2.cvtColor(frame, cv2.COLOR_BGR2RGBA) # because Bokeh expects a RGBA image
#         frame=cv2.flip(frame, -1) # because Bokeh flips vertically
        frame=cv2.flip(frame, 0) # because Bokeh flips vertically
        width=frame.shape[1]
        height=frame.shape[0]
        p = figure(x_range=(0,width), y_range=(0,height), output_backend="webgl", width=width, height=height)
        self._myImage = p.image_rgba(image=[frame], x=0, y=0, dw=width, dh=height)
        show(p, notebook_handle=True)
    
    def update_display(self, frame):
        assert frame is not None
        frame=cv2.cvtColor(frame, cv2.COLOR_BGR2RGBA)
#         frame=cv2.flip(frame, -1)
        frame=cv2.flip(frame, 0) 
        self._myImage.data_source.data['image']=[frame]
        push_notebook()

In [ ]:
def detection_result_process(frame, classes, class_IDs, scores, bounding_boxes, hand_cnt, no_hand_cnt, start_trans, in_trans, curr_item_cnt, max_item_cnt, pre_msg, pre_msg2):
    thres = 0.45
    if class_IDs is not None and len(class_IDs) == 1:
            if len(class_IDs[0]) >= 1:
                hand = False
                item_cnt = [0, 0, 0]
                for i in range(len(class_IDs[0])):
                    if scores[0][i] > thres:
                        class_ID = int(class_IDs[0][i])
                        score = float(scores[0][i])
                        bounding_box = bounding_boxes[0][i]
                        # print('class_ID:', class_ID, 'score:', score, 'bounding_box:', bounding_box)
                        xmin, ymin, xmax, ymax = [int(x) for x in bounding_box]
                        cv2.rectangle(frame, (xmin, ymin), (xmax, ymax), color=(0, 255, 0), thickness=1)
                        cv2.putText(frame, str(classes[class_ID])+':'+str(round(score, 2)), (xmin+10, ymin+10), fontFace=cv2.FONT_HERSHEY_SIMPLEX, fontScale=0.5, thickness=1, color=(255, 255, 255))
                        
                        if class_ID == 3:
                            hand = True
                        else:
                            item_cnt[class_ID] += 1
                        
                for i in range(len(item_cnt)):
                    item_cnt[i] = min(item_cnt[i], max_item_cnt[i])
                        
                if not hand:
                    no_hand_cnt += 1
                else:
                    hand_cnt += 1
                    
                if hand_cnt >= 3:
                    start_trans = True
                    in_trans = True
                    hand_cnt = 0
                    no_hand_cnt = 0
                elif no_hand_cnt >= 3:
                    start_trans = False
                    hand_cnt = 0
                    no_hand_cnt = 0
                    
                if start_trans:
                    msg = 'Start Transaction'
                else:
                    msg = 'End Transaction'
                    if in_trans:
                        for i in range(len(item_cnt)):
                            change_item = item_cnt[i]-curr_item_cnt[i]
                            if change_item != 0:
                                char = ''
                                if change_item > 0:
                                    char = '+'
                                msg += ' '+classes[i]+': '+char+str(change_item)
                        in_trans = False
                        curr_item_cnt = item_cnt
                    else:
                        msg = pre_msg
                    
                    if not hand:
                        for i in range(len(item_cnt)):
                            curr_item_cnt[i] = max(curr_item_cnt[i], item_cnt[i])

                msg2 = ''
                for i in range(len(curr_item_cnt)):
                    msg2 += classes[i]+': '+str(curr_item_cnt[i])+' '
                
                cv2.putText(frame, msg, (40, 40), fontFace=cv2.FONT_HERSHEY_SIMPLEX, fontScale=0.8, thickness=2, color=(255, 255, 255))
                cv2.putText(frame, msg2, (40, 60), fontFace=cv2.FONT_HERSHEY_SIMPLEX, fontScale=0.5, thickness=1, color=(255, 255, 255))
    return frame, hand_cnt, no_hand_cnt, start_trans, in_trans, curr_item_cnt, msg, msg2

In [ ]:
"""
need add rtmp support for sagemaker, please follow this page
https://answers.opencv.org/question/180776/build-opencv-with-ffmpeg-support/
"""
url="rtmp://ec2-54-222-130-39.cn-north-1.compute.amazonaws.com.cn/live/123456"

vcap = cv2.VideoCapture(url)
player = VideoPlayer()

hand_cnt = 0
no_hand_cnt = 0
start_trans = False
in_trans = False
curr_item_cnt = [0, 0, 0]
max_item_cnt = [4, 1, 1]
pre_msg = ''
pre_msg2 = ''

while(True):
    # Capture frame-by-frame
    read_start = time.time()
    ret, frame = vcap.read()
    ret, frame = vcap.read()
    ret, frame = vcap.read()
    read_end = time.time()
    #print('read time:', read_end-read_start)

    if frame is not None:
        start = time.time()
        frame = cv2.flip(frame, -1)
        
        # save image
        #filename = 'frame_'+str(time.time())+'.jpg'
        #cv2.imwrite(filename, frame)

        # use SageMaker
        #with open(filename, 'rb') as image:
        #    f = image.read()
        #    b = bytearray(f)
        #result = sagemaker_endpoint.predict(b)
        #print(json.loads(result))
        
        # use GluonCV
        detect_start = time.time()
        #class_IDs, scores, bounding_boxes = objectDetection.detect_image(frame)
        class_IDs, scores, bounding_boxes = objectDetection.detect_image_yolo(frame)
        #print('class_IDs:', class_IDs)
        #print('scores:', scores)
        #print('bounding_boxes:', bounding_boxes)
        detect_end = time.time()
        #print('detect time:', detect_end-detect_start)
        
        frame, hand_cnt, no_hand_cnt, start_trans, in_trans, curr_item_cnt, msg, msg2 = detection_result_process(frame, objectDetection.classes, class_IDs, scores, bounding_boxes, hand_cnt, no_hand_cnt, start_trans, in_trans, curr_item_cnt, max_item_cnt, pre_msg, pre_msg2)
        #print(time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(detect_end)), 'msg:', msg)
        #print(time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(detect_end)), 'msg2:', msg2)
        pre_msg = msg
        pre_msg2 = msg2
        
        # Display the resulting frame
        #cv2.imwrite(filename, frame)
        player(frame)
        end = time.time()
        #print('all time:', end-start)
        
        #t = 1
        #time.sleep(t)
        #vcap.set(cv2.CAP_PROP_POS_FRAMES, t*25)
        
        #t = 0.2
        #sleep_time = t-(end-start)
        #if sleep_time >= 0:
        #    time.sleep(sleep_time)
        #    vcap.set(cv2.CAP_PROP_POS_FRAMES, sleep_time*25)
    
        #skip_frame = max(0, 25-int(1/(end-start)))
        #print('skip_frame:', skip_frame)
        #if skip_frame > 0:
        #    vcap.set(cv2.CAP_PROP_POS_FRAMES, skip_frame)

        #break

        # Press q to close the video windows before it ends if you want
        #if cv2.waitKey(22) & 0xFF == ord('q'):
        #    break
    else:
        print("Frame is None")
        break

# When everything done, release the capture
vcap.release()
print("Video stop")